In [1]:
%%pyspark project.spark.compatibility
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col, length, when
import re


df = spark.read.option("header", "true").csv("raw_path")

Executing for connection type: SPARK_GLUE, connection name: project.spark.compatibility
Creating Glue session...
Create session for connection: project.spark.compatibility


'Session 3vao8u7vieu8sw-5c8deb16-394b-4c45-8563-57e9e7790299 has been created.'

<sagemaker_studio_dataengineering_sessions.sagemaker_base_session_manager.common.debugging_utils.SessionInfoTableDisplay object>

Session created for connection: project.spark.compatibility.

Connection: project.spark.compatibility | Run start time: 2025-12-10 06:16:11.036343 | Run duration : 0:01:03.160936s.


In [27]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df.show()

Connection: project.spark.compatibility | Run start time: 2025-12-10 06:20:10.555955 | Run duration : 0:00:00.884972s.


In [7]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment

def clean_text_combined(text):
    if text is None:
        return ""
    
    try:
        # Keep only alphanumeric characters and spaces
        return re.sub(r'[^a-zA-Z0-9가-힣\s]', '', text).strip()
    except:
        return text


Connection: project.spark.compatibility | Run start time: 2025-12-10 06:17:43.050730 | Run duration : 0:00:03.792739s.


In [8]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
clean_text_combined_udf = udf(clean_text_combined, StringType())
df_final = df.withColumn("contents_final", clean_text_combined_udf(col("contents"))) \
            .filter(length(col("contents_final")) > 30)


Connection: project.spark.compatibility | Run start time: 2025-12-10 06:17:46.849088 | Run duration : 0:00:03.430048s.


In [9]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_cleaned_two=df_final.select('title', 'contents_final', 'label', 'frequency')


Connection: project.spark.compatibility | Run start time: 2025-12-10 06:17:50.286279 | Run duration : 0:00:03.635428s.


In [10]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_cleaned_two.count()


995
Connection: project.spark.compatibility | Run start time: 2025-12-10 06:17:53.926469 | Run duration : 0:00:07.221227s.


In [26]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_cleaned_two.show()

Connection: project.spark.compatibility | Run start time: 2025-12-10 06:20:01.934164 | Run duration : 0:00:01.345899s.


In [12]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered = df_cleaned_two.filter(col("label").isin(["Y", "N"]))


Connection: project.spark.compatibility | Run start time: 2025-12-10 06:18:10.247736 | Run duration : 0:00:03.599086s.


In [13]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered.count()


985
Connection: project.spark.compatibility | Run start time: 2025-12-10 06:18:13.853702 | Run duration : 0:00:09.360877s.


In [14]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered_final = df_filtered.filter(col("label").isin(["Y", "N"])) \
    .withColumn("label", when(col("label") == "Y", 1).otherwise(0)) \
    .withColumn("frequency", col("frequency").cast("int"))


Connection: project.spark.compatibility | Run start time: 2025-12-10 06:18:23.220741 | Run duration : 0:00:03.458714s.


In [25]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
df_filtered_final.show()

Connection: project.spark.compatibility | Run start time: 2025-12-10 06:19:51.763155 | Run duration : 0:00:00.853800s.


In [16]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
# label별로 분리
df_label_0 = df_filtered_final.filter(col("label") == 0)
df_label_1 = df_filtered_final.filter(col("label") == 1)

# 각각 split
train_0, test_0 = df_label_0.randomSplit([0.7, 0.3], seed=42)
train_1, test_1 = df_label_1.randomSplit([0.7, 0.3], seed=42)

# 합치기
train_df = train_0.union(train_1).orderBy(rand(seed=42))
test_df = test_0.union(test_1).orderBy(rand(seed=42))


Connection: project.spark.compatibility | Run start time: 2025-12-10 06:18:33.951987 | Run duration : 0:00:03.950569s.


In [24]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
train_df.show()

Connection: project.spark.compatibility | Run start time: 2025-12-10 06:19:44.962030 | Run duration : 0:00:00.796693s.


In [23]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
test_df.show()

Connection: project.spark.compatibility | Run start time: 2025-12-10 06:19:37.470781 | Run duration : 0:00:00.785710s.


In [19]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
train_df.count()


726
Connection: project.spark.compatibility | Run start time: 2025-12-10 06:18:55.830221 | Run duration : 0:00:09.949064s.


In [20]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
test_df.count()


259
Connection: project.spark.compatibility | Run start time: 2025-12-10 06:19:05.785911 | Run duration : 0:00:09.307526s.


In [21]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
train_df.write.option("header",True) \
 .csv("train_path")



Connection: project.spark.compatibility | Run start time: 2025-12-10 06:19:15.099572 | Run duration : 0:00:10.985863s.


In [22]:
%%pyspark project.spark.compatibility
# Enter your code at the start of this line to replace this comment
test_df.write.option("header",True) \
 .csv("test_path")



Connection: project.spark.compatibility | Run start time: 2025-12-10 06:19:26.092591 | Run duration : 0:00:09.064317s.


In [ ]:
%%pyspark default.spark
# Enter your code at the start of this line to replace this comment